In [1]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

from datetime import datetime
import fxcmpy
from sklearn import svm
import time

In [2]:
con = fxcmpy.fxcmpy(config_file='fxcm.cfg')

In [3]:
fxcmpy.__version__

'1.1.16'

In [4]:
to_show = ['tradeId', 'amountK','currency','grossPL','isBuy']

In [5]:
lags = 3

In [6]:
model = svm.SVC(C=100, probability=True)

In [7]:
def generate_features(df, lags):
    df['Returns'] = np.log(df['Mid'] /df['Mid'].shift(1))
    print(df)
    cols = []
    for lag in range(1, lags +1):
        col = 'lag_%s' % lag
        print("col = ", col)
        df[col] = np.sign(df['Returns'].shift(lag))
        cols.append(col)
    df.dropna(inplace=True)
    return df, cols

In [8]:
candles = con.get_candles('USD/JPY', period='m1', number=500)

In [9]:
data = pd.DataFrame(candles[['askclose', 'bidclose']].mean(axis=1),
                   columns=['Mid'])

In [10]:
data, cols = generate_features(data, lags)
data.tail()

                          Mid   Returns
date                                   
2019-05-02 17:42:00  111.4970       NaN
2019-05-02 17:43:00  111.5025  0.000049
2019-05-02 17:44:00  111.4970 -0.000049
2019-05-02 17:45:00  111.4890 -0.000072
2019-05-02 17:46:00  111.4950  0.000054
2019-05-02 17:47:00  111.4945 -0.000004
2019-05-02 17:48:00  111.4950  0.000004
2019-05-02 17:49:00  111.4925 -0.000022
2019-05-02 17:50:00  111.4940  0.000013
2019-05-02 17:51:00  111.4960  0.000018
2019-05-02 17:52:00  111.4960  0.000000
2019-05-02 17:53:00  111.4950 -0.000009
2019-05-02 17:54:00  111.4875 -0.000067
2019-05-02 17:55:00  111.4910  0.000031
2019-05-02 17:56:00  111.4945  0.000031
2019-05-02 17:57:00  111.4930 -0.000013
2019-05-02 17:58:00  111.4900 -0.000027
2019-05-02 17:59:00  111.4835 -0.000058
2019-05-02 18:00:00  111.4685 -0.000135
2019-05-02 18:01:00  111.4690  0.000004
2019-05-02 18:02:00  111.4685 -0.000004
2019-05-02 18:03:00  111.4760  0.000067
2019-05-02 18:04:00  111.4830  0.000063


,Mid,Returns,lag_1,lag_2,lag_3
date,,,,,
2019-05-03 01:58:00,111.4295,-0.000036,1.0,1.0,-1.0
2019-05-03 01:59:00,111.4330,0.000031,-1.0,1.0,1.0
2019-05-03 02:00:00,111.4330,0.000000,1.0,-1.0,1.0
2019-05-03 02:01:00,111.4330,0.000000,0.0,1.0,-1.0
2019-05-03 02:02:00,111.4275,-0.000049,0.0,0.0,1.0


In [11]:
model.fit(data[cols], np.sign(data['Returns']))

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
ticks=0
position=0
tick_data=pd.DataFrame()
tick_resam=pd.DataFrame()

In [13]:
def automated_trading(data,df):
    global lags,model,ticks
    global tick_data,tick_resam,to_show
    global position
    ticks += 1
    t =datetime.now()
    if ticks % 5==0 :
        print('%3d|%s| %7.5f | %7.5f' % (ticks, str(t.time()),
                                         data['Rates'][0],data['Rates'][1]))
    
    #collecting tick data    
    tick_data=tick_data.append(pd.DataFrame(
        {'Bid':data['Rates'][0],'Ask':data['Rates'][1],
         'High':data['Rates'][2],'Low':data['Rates'][3]},
         index=[t]))  
    
    #resampling tick data
    tick_resam=tick_data[['Bid','Ask']].resample('60s',label='right').last().ffill()
    tick_resam['Mid'] = tick_resam.mean(axis=1)
    time.sleep(1)
    if len(tick_resam) > lags + 2:
        #generate a signal
        tick_resam, cols = generate_features(tick_resam, lags)
        tick_resam['Prediction'] = model.predict(tick_resam[cols])
        #entering a long position 
        if tick_resam['Prediction'].iloc[-2] >= 0 and position == 0:
            print('going long (for the first time)')
            position = 1
            order = con.create_market_buy_order('USD/JPY', 25)
            trade = True 
        elif tick_resam['Prediction'].iloc[-2] >= 0 and position == -1:
            print('going long')
            position = 1
            order = con.create_market_buy_order('USD/JPY', 50)
            trade = True
        #entering a short positon 
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 0:
            print('going short (for the first time)')
            position = -1
            order = con.create_market_buy_order('USD/JPY', 25)
            trade = True
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 1:
            print('going short')
            position = -1
            order = con.create_market_buy_order('USD/JPY', 50)
            trade = True

In [14]:
con.subscribe_market_data('USD/JPY', (automated_trading,))

  5|14:04:52.937363| 111.42200 | 111.43400
 10|14:05:06.253697| 111.42300 | 111.43500
 15|14:06:24.588550| 111.42300 | 111.43500
 20|14:06:43.634723| 111.42100 | 111.43400
 25|14:07:36.165059| 111.41500 | 111.42900


In [15]:
con.unsubscribe_market_data('USD/JPY')

                         Bid      Ask       Mid       Returns
2019-05-03 14:05:00  111.423  111.434  111.4285           NaN
2019-05-03 14:06:00  111.422  111.435  111.4285  2.220446e-16
2019-05-03 14:07:00  111.420  111.434  111.4270 -1.346164e-05
2019-05-03 14:08:00  111.415  111.428  111.4215 -4.936089e-05
2019-05-03 14:09:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:10:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:11:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:12:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:13:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:14:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:15:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:16:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:17:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:18:00  111.415  111.428  111.4215  0.000000e+00
2019-05-03 14:19:00  111.415  111.428  111.4215  0.000000e+00
2019-05-